# Pyrosm

Meant to be more offline friendly than osmnx. 

It replaces the Overpass api with local .pbt files

Unfortunetly it changes the API too much and it my experience not managed to get it to work for a subregion Lancashire.

Lancashire is not present in subregions even though it can be downloaded online

Docs: https://pyrosm.readthedocs.io/en/latest/reference.html#pyrosm.pyrosm.OSM

Lancashire @GeoFabrik: https://download.geofabrik.de/europe/united-kingdom/england/lancashire.html

Leeds University: https://udsleeds.github.io/openinfra/articles/osm-python.html

In [ ]:
'''
Get the data from Pyrosm 
'''
import pyrosm as ox

osm_path = "./osm"

def download_region(region="Lancashire"):
    
    available_places = pyrosm.data.available
    print(available_places['subregions'].keys())
    print(f"{region} available from providers:", "england" in available_places['subregions'])

if __name__ == "__main__":
    download_region();

In [ ]:
'''
Pyrosm implementation, but not in use

Issue: the Lancashire only .pbf is probably too narrow for Pyrosm and when we attempt to load it, throws 'id' issues
'''

import geopandas as gpd
import pyrosm
from pyrosm import OSM
import folium

pbf_path = "./osm/lancashire-latest.osm.pbf"
activity_path = "./out/osm/activity.geojson"
out_map = "./out/osm/route_map.html"
crs_uk = 'EPSG:27700'  # UK Ordnance Survey CRS
crs_geo = 'EPSG:4326'  # WGS84 for geographic operations

def read_geojson(geojson_path: str) -> gpd.GeoDataFrame:
    """Read and validate GeoJSON file with proper CRS handling."""
    try:
        # Convert to geographic CRS for OSM operations
        gdf = gpd.read_file(geojson_path).to_crs(crs_geo)
        if gdf.empty:
            raise ValueError("GeoJSON file is empty")
        return gdf
    except Exception as e:
        raise RuntimeError(f"Failed to read GeoJSON: {str(e)}") from e

def get_route_geometry(gdf: gpd.GeoDataFrame) -> gpd.GeoSeries:
    """Extract and validate route geometry."""
    if not isinstance(gdf, gpd.GeoDataFrame):
        raise TypeError("Input must be a GeoDataFrame")
    return gdf.geometry.iloc[0]

def calculate_route_bbox(geometry: gpd.GeoSeries, buffer_meters: int = 2000) -> tuple:
    """Calculate buffered bounding box in WGS84."""
    try:
        #
        geo_geom = geometry
        west, south, east, north = geo_geom.bounds
        
        # Convert buffer meters to degrees (approximate)
        buffer_deg = buffer_meters / 111000  # 1° ≈ 111km
        return [
            west - buffer_deg,
            south - buffer_deg,
            east + buffer_deg,
            north + buffer_deg
        ]
    except Exception as e:
        raise RuntimeError(f"Bounding box calculation failed: {str(e)}") from e

def verify_pbf_coverage(pbf_path: str, bbox: tuple):
    """Check if PBF file contains the target area."""
    osm = OSM(pbf_path)
    pbf_bbox = osm.bounding_box

    print("PBF bbox", pbf_bbox)
    
    # Check if request bbox is within PBF bbox
    if not (bbox[0] >= pbf_bbox[0] and 
            bbox[1] >= pbf_bbox[1] and
            bbox[2] <= pbf_bbox[2] and
            bbox[3] <= pbf_bbox[3]):
        raise ValueError(f"PBF file {pbf_path} doesn't cover target area {bbox}")

def get_osm_data(pbf_path: str, bbox: tuple) -> tuple:
    """Fetch OSM network data within specified bounds."""
    try:
        osm = OSM(pbf_path, bounding_box=bbox)
        return osm.get_network(nodes=True, network_type="walking")
    except Exception as e:
        raise RuntimeError(f"OSM data loading failed: {str(e)}") from e

def create_map(route_geom: gpd.GeoSeries, edges: gpd.GeoDataFrame) -> folium.Map:
    """Create Folium map with OSM context and route overlay."""
    try:
        # Convert route geometry to geographic CRS
        route_geo = route_geom.to_crs(crs_geo)
        centroid = route_geo.centroid
        m = folium.Map(
            location=[centroid.y, centroid.x],
            zoom_start=14,
            tiles="CartoDB positron"
        )

        # Add OSM edges
        for _, row in edges.iterrows():
            folium_coords = [(y, x) for x, y in row.geometry.coords]
            folium.PolyLine(
                folium_coords,
                color="#cccccc",
                weight=1.5,
                opacity=0.7
            ).add_to(m)

        # Add activity route
        route_coords = [(y, x) for x, y in route_geo.coords]
        folium.PolyLine(
            route_coords,
            color="#ff0000",
            weight=4,
            opacity=0.9,
            tooltip="Activity Route"
        ).add_to(m)

        # Add markers
        start, end = route_coords[0], route_coords[-1]
        folium.Marker(start, icon=folium.Icon(color="green")).add_to(m)
        folium.Marker(end, icon=folium.Icon(color="blue")).add_to(m)

        return m

    except Exception as e:
        raise RuntimeError(f"Map creation failed: {str(e)}") from e

def main():
    """Main workflow execution."""
    try:
        # Load and validate data
        gdf = read_geojson(activity_path)
        route_geom = get_route_geometry(gdf)
        
        # Calculate bounds and fetch OSM data
        bbox = calculate_route_bbox(route_geom)

        # Verify PBF coverage
        verify_pbf_coverage(pbf_path, bbox)
        
        nodes, edges = get_osm_data(pbf_path, bbox)
        print("nodes/edges", (nodes, edges))
        # Create and save map
        # m = create_map(route_geom, edges)
        # m.save(out_map)
        print(f"Map successfully saved to {out_map}")
        
    except Exception as e:
        print(f"Error: {str(e)}")
        raise

if __name__ == "__main__":
    main()